# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Laboratorio 3: Deep Learning

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169

**Curso:** Data Science  
**Sección:** 10  

---

Guatemala, 08 de agosto de 2024


<div style="text-align: center;">
    <h1>✍️ Discusión / Cálculos - [Deep Learning] ✍️</h1>
</div>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv
import statsmodels.api as sm
import matplotlib.pyplot as plt
from autoviz.AutoViz_Class import AutoViz_Class
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
%matplotlib inline

### 📚 Carga de datasets 

In [2]:
csvPath = 'Consumo/dataConsumo.csv'
dataConsumo = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

csvPath = 'Importacion/dataImportacion.csv'
dataImportacion = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

csvPath = 'PreciosPromedioNacionales/dataPrecios.csv'
dataPrecios = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

✅ Archivo cargado: Consumo/dataConsumo.csv
✅ Archivo cargado: Importacion/dataImportacion.csv
✅ Archivo cargado: PreciosPromedioNacionales/dataPrecios.csv


In [3]:
serieConsumoDiesel = dataConsumo[['Fecha', 'Diesel']]
serieImportacionRegular = dataImportacion[['Fecha', 'Gasolina regular']]
seriePreciosSuper = dataPrecios[['FECHA', 'Superior GTQ/GALON']]
serieConsumoDiesel.set_index('Fecha', inplace=True)
serieConsumoDiesel.sort_index(inplace=True)
serieImportacionRegular.set_index('Fecha', inplace=True)
serieImportacionRegular.sort_index(inplace=True)
seriePreciosSuper.set_index('FECHA', inplace=True)
seriePreciosSuper.sort_index(inplace=True)

In [5]:
def pruebaDickeyFuller(serieTiempo, descripcion):
    print(f"📊 Prueba de Dickey-Fuller Aumentada para: {descripcion}")
    resultado = adfuller(serieTiempo)
    print(f'Estadístico ADF: {resultado[0]}')
    print(f'p-valor: {resultado[1]}')
    print('Valores críticos:')
    for key, value in resultado[4].items():
        print(f'  {key}: {value}')
    print('-----------------------------------------------------')
    return resultado[1] 

print('-----------------------------------------------------')
pruebaDickeyFuller(serieConsumoDiesel['Diesel'], 'Diesel')
pruebaDickeyFuller(serieImportacionRegular['Gasolina regular'], 'Regular')
pruebaDickeyFuller(seriePreciosSuper['Superior GTQ/GALON'], 'Superior')

-----------------------------------------------------
📊 Prueba de Dickey-Fuller Aumentada para: Diesel
Estadístico ADF: 0.14523982117810716
p-valor: 0.9690175028779467
Valores críticos:
  1%: -3.453922368485787
  5%: -2.871918329081633
  10%: -2.5723001147959184
-----------------------------------------------------
📊 Prueba de Dickey-Fuller Aumentada para: Regular
Estadístico ADF: 0.8288289928204905
p-valor: 0.992087554110633
Valores críticos:
  1%: -3.4547128138328875
  5%: -2.8722649771800155
  10%: -2.5724850011573914
-----------------------------------------------------
📊 Prueba de Dickey-Fuller Aumentada para: Superior
Estadístico ADF: -2.9399220843939617
p-valor: 0.040898674582741115
Valores críticos:
  1%: -3.435401880796999
  5%: -2.863770985550096
  10%: -2.567957791647768
-----------------------------------------------------


0.040898674582741115

In [6]:
def detectar_valores_invalidos(serieTemporal, descripcion):
    serieTemporal = pd.to_numeric(serieTemporal, errors='coerce')  # Convertir a numérico y forzar NaN si hay errores
    n_nan = serieTemporal.isna().sum()
    n_no_positivos = (serieTemporal <= 0).sum()

    print(f"🔍 Detección de valores inválidos en la serie: {descripcion}")
    print(f"  Valores NaN: {n_nan}")
    print(f"  Valores no positivos: {n_no_positivos}")
    
    serieTemporal_sin_nan = serieTemporal.dropna()
    print(f"  Serie temporal sin valores NaN tiene {len(serieTemporal_sin_nan)} elementos.")
    
    return serieTemporal_sin_nan, n_nan, n_no_positivos

def varianza_estable(serieTemporal):
    n = len(serieTemporal)
    partes = np.array_split(serieTemporal, 3)
    varianzas = [np.var(parte) for parte in partes]
    return np.allclose(varianzas[0], varianzas[1], rtol=0.1) and np.allclose(varianzas[1], varianzas[2], rtol=0.1)

def aplicar_transformacion_logaritmica(serieTemporal):
    if (serieTemporal > 0).all():
        return np.log(serieTemporal)
    else:
        print("⚠️ La serie contiene valores no positivos, no se puede aplicar la transformación logarítmica.")
        return serieTemporal

def analizar_y_transformar_serie(serieTemporal, descripcion):
    print(f"🔍 Analizando la serie: {descripcion}")
    
    serieTemporal, n_nan, n_no_positivos = detectar_valores_invalidos(serieTemporal, descripcion)
    
    estacionaria = False
    intentos = 0
    max_intentos = 10

    serieTemporal = aplicar_transformacion_logaritmica(serieTemporal)
    
    while not estacionaria and intentos < max_intentos:
        intentos += 1
        p_valor = pruebaDickeyFuller(serieTemporal, descripcion)
        if p_valor < 0.05:
            if varianza_estable(serieTemporal):
                estacionaria = True
                print(f"✅ La serie {descripcion} es estacionaria en media y varianza.")
                break
            else:
                print(f"❌ La varianza no es estable en la serie {descripcion}.")
        else:
            print(f"❌ La serie {descripcion} no es estacionaria (p-valor: {p_valor}).")
        
        serieTemporal = serieTemporal.diff().dropna()
        print(f"🔄 Aplicando diferenciación a {descripcion}. Intento {intentos}.")
    
    if not estacionaria:
        print(f"⚠️ No se pudo hacer la serie {descripcion} estacionaria después de {max_intentos} intentos.")
        print("📈 Utilizando un modelo ARIMA para tratar con la no estacionariedad.")
        modelo_arima = ARIMA(serieTemporal, order=(1, 1, 1))
        modelo_arima_fit = modelo_arima.fit()
        print(modelo_arima_fit.summary())
    else:
        modelo_arima_fit = None
        
    return serieTemporal, modelo_arima_fit

# Asegúrate de que estas variables estén definidas correctamente
# Reemplaza con tus series temporales reales
serieTemporalD, modelo_arima_diesel = analizar_y_transformar_serie(serieConsumoDiesel['Diesel'], 'Diesel')
# print("--------------------------------------------------------------------------------------------------------")
serieTemporalR, modelo_arima_regular = analizar_y_transformar_serie(serieImportacionRegular['Gasolina regular'], 'Regular')
# print("--------------------------------------------------------------------------------------------------------")
serieTemporalS, modelo_arima_super = analizar_y_transformar_serie(seriePreciosSuper['Superior GTQ/GALON'], 'Superior')


🔍 Analizando la serie: Diesel
🔍 Detección de valores inválidos en la serie: Diesel
  Valores NaN: 0
  Valores no positivos: 0
  Serie temporal sin valores NaN tiene 293 elementos.
📊 Prueba de Dickey-Fuller Aumentada para: Diesel
Estadístico ADF: -0.44918144412882166
p-valor: 0.901557753919378
Valores críticos:
  1%: -3.453922368485787
  5%: -2.871918329081633
  10%: -2.5723001147959184
-----------------------------------------------------
❌ La serie Diesel no es estacionaria (p-valor: 0.901557753919378).
🔄 Aplicando diferenciación a Diesel. Intento 1.
📊 Prueba de Dickey-Fuller Aumentada para: Diesel
Estadístico ADF: -6.9885899777221345
p-valor: 7.851113111171977e-10
Valores críticos:
  1%: -3.453922368485787
  5%: -2.871918329081633
  10%: -2.5723001147959184
-----------------------------------------------------
❌ La varianza no es estable en la serie Diesel.
🔄 Aplicando diferenciación a Diesel. Intento 2.
📊 Prueba de Dickey-Fuller Aumentada para: Diesel
Estadístico ADF: -9.12656830456

### 🧪 Division train y test

In [12]:
# Dividir los datos en 70% entrenamiento, 15% validación y 15% prueba
def dividir_datos(serie, train_size=0.7, val_size=0.15):
    n = len(serie)
    train_end = int(n * train_size)
    val_end = int(n * (train_size + val_size))
    
    train = serie[:train_end]
    val = serie[train_end:val_end]
    test = serie[val_end:]
    
    return train, val, test

In [13]:
def supervisada(serie, retrasos=1):
    serie_x = []
    serie_y = []
    for i in range(len(serie) - retrasos):
        valor = serie[i:(i + retrasos), 0]
        valor_sig = serie[i + retrasos, 0]
        serie_x.append(valor)
        serie_y.append(valor_sig)
    return np.array(serie_x), np.array(serie_y)


In [18]:
def toDataframe(serie):
    if not isinstance(serie, pd.DataFrame):
        print("📝 Convirtiendo a DataFrame")
        return serie.to_frame()
    return serie

In [19]:
# Preparar y escalar datos
scaler = StandardScaler()

# Diesel
print("⛽ Preparando datos Diesel")
serieTemporalD = toDataframe(serieTemporalD)
serieTemporalDScaled = scaler.fit_transform(serieTemporalD)
serieTemporalDScaled = pd.DataFrame(serieTemporalDScaled, index=serieTemporalD.index, columns=serieTemporalD.columns)

trainL3D, valL3D, testL3D = dividir_datos(serieTemporalDScaled)

x_trainD, y_trainD = supervisada(trainL3D.values)
x_valD, y_valD = supervisada(valL3D.values)
x_testD, y_testD = supervisada(testL3D.values)

# Gasolina Regular
print("⛽ Preparando datos Gasolina Regular")
serieTemporalR = toDataframe(serieTemporalR)
serieTemporalRScaled = scaler.fit_transform(serieTemporalR)
serieTemporalRScaled = pd.DataFrame(serieTemporalRScaled, index=serieTemporalR.index, columns=serieTemporalR.columns)

trainL3R, valL3R, testL3R = dividir_datos(serieTemporalRScaled)

x_trainR, y_trainR = supervisada(trainL3R.values)
x_valR, y_valR = supervisada(valL3R.values)
x_testR, y_testR = supervisada(testL3R.values)

# Gasolina Superior
print("⛽ Preparando datos Gasolina Superior")
serieTemporalS = toDataframe(serieTemporalS)
serieTemporalSScaled = scaler.fit_transform(serieTemporalS)
serieTemporalSScaled = pd.DataFrame(serieTemporalSScaled, index=serieTemporalS.index, columns=serieTemporalS.columns)

trainL3S, valL3S, testL3S = dividir_datos(serieTemporalSScaled)

x_trainS, y_trainS = supervisada(trainL3S.values)
x_valS, y_valS = supervisada(valL3S.values)
x_testS, y_testS = supervisada(testL3S.values)


⛽ Preparando datos Diesel
⛽ Preparando datos Gasolina Regular
⛽ Preparando datos Gasolina Superior


### 🌀 Transformando a 3D

In [21]:
# Función para transformar matrices 2D en matrices 3D para LSTM
def transformar_a_3D(x):
    return np.reshape(x, (x.shape[0], 1, 1))

# Usando la función en los conjuntos de datos
def preparar_datos_lstm(x_train, y_train, x_val, y_val, x_test, y_test):
    print("🌀 Transformando a 3D...")    
    x_train_3D = transformar_a_3D(x_train)
    x_val_3D = transformar_a_3D(x_val)
    x_test_3D = transformar_a_3D(x_test)
    
    return x_train_3D, y_train, x_val_3D, y_val, x_test_3D, y_test

# Diesel
x_trainD_3D, y_trainD, x_valD_3D, y_valD, x_testD_3D, y_testD = preparar_datos_lstm(x_trainD, y_trainD, x_valD, y_valD, x_testD, y_testD)
print("⛽ Datos de diesel transformados.")  

# Gasolina Regular
x_trainR_3D, y_trainR, x_valR_3D, y_valR, x_testR_3D, y_testR = preparar_datos_lstm(x_trainR, y_trainR, x_valR, y_valR, x_testR, y_testR)
print("⛽ Datos de gasolina regular transformados.")

# Gasolina Superior
x_trainS_3D, y_trainS, x_valS_3D, y_valS, x_testS_3D, y_testS = preparar_datos_lstm(x_trainS, y_trainS, x_valS, y_valS, x_testS, y_testS)
print("⛽ Datos de gasolina superior transformados.")

🌀 Transformando a 3D...
⛽ Datos de diesel transformados.
🌀 Transformando a 3D...
⛽ Datos de gasolina regular transformados.
🌀 Transformando a 3D...
⛽ Datos de gasolina superior transformados.
